# Eight Puzzle Solver
We solve the classic 8-puzzle using Breadth-First Search (BFS). Each move slides the blank tile (`0`) into an adjacent position.

### Understanding the 8-Puzzle and BFS

The 8-puzzle is a classic sliding puzzle that consists of a 3x3 grid with 8 numbered tiles and one blank space. The goal is to rearrange the tiles from a random starting configuration into a sorted order.

#### Breadth-First Search (BFS)

BFS is an algorithm for traversing or searching tree or graph data structures. It starts at a selected node (the "root") and explores all of the neighbor nodes at the present depth prior to moving on to the nodes at the next depth level.

#### Why BFS for the 8-Puzzle?

- **Completeness**: If a solution exists, BFS is guaranteed to find it.
- **Optimality**: BFS always finds the shortest path (in terms of the number of moves) from the start state to the goal state. This is because it explores the puzzle state layer by layer, so it finds the goal at the shallowest possible depth.

#### How It Works

1.  **Initialization**: Start with a queue and add the initial puzzle state to it. Use a set or dictionary to keep track of visited states to avoid cycles and redundant work.
2.  **Iteration**: While the queue is not empty, dequeue the current state.
3.  **Goal Test**: Check if the current state is the goal state. If it is, we're done.
4.  **Expansion**: If not, generate all valid neighboring states (by sliding the blank tile up, down, left, or right).
5.  **Enqueue**: For each new, unvisited neighbor, mark it as visited and add it to the back of the queue. We also store its "parent" state so we can reconstruct the path later.

In [ ]:
from collections import deque

# State is a tuple of 9 numbers laid out row-wise
start_state = (1, 2, 3,
               4, 0, 6,
               7, 5, 8)
goal_state = (1, 2, 3,
              4, 5, 6,
              7, 8, 0)

def show_state(state):
    """Print the puzzle as a 3x3 board."""
    for row in range(0, 9, 3):
        slice_ = state[row:row + 3]
        print(" ".join(str(num) for num in slice_))

### Code: State Representation and Setup

This cell defines how we represent the puzzle and sets up the initial and final states.

- **State Representation**: A `tuple` of 9 numbers is used to represent the board. A tuple is used because it is immutable, which allows it to be used as a key in a dictionary or an element in a set (essential for our `parents` map). The numbers are laid out row by row. The blank space is represented by `0`.
- **`start_state`**: The initial configuration of the puzzle.
- **`goal_state`**: The target configuration we want to reach.
- **`show_state(state)`**: A simple helper function to print the board in a human-readable 3x3 format.

In [ ]:
def neighbors(state):
    """Generate all states by sliding a tile into the blank."""
    zero_index = state.index(0)
    row, col = divmod(zero_index, 3)
    swaps = []
    if row > 0:
        swaps.append(zero_index - 3)
    if row < 2:
        swaps.append(zero_index + 3)
    if col > 0:
        swaps.append(zero_index - 1)
    if col < 2:
        swaps.append(zero_index + 1)

    for swap_index in swaps:
        new_state = list(state)
        new_state[zero_index], new_state[swap_index] = new_state[swap_index], new_state[zero_index]
        yield tuple(new_state)

### Code: Generating Neighboring States

The `neighbors` function is crucial for exploring the search space. Given a puzzle state, it generates all possible next states.

1.  **Find the Blank**: It first locates the index of the blank tile (`0`).
2.  **Determine Valid Moves**: Based on the row and column of the blank tile, it determines which moves are possible (e.g., you can't move up if the blank is in the top row).
3.  **Generate New States**: For each valid move, it creates a new list from the current state, swaps the blank tile with the adjacent tile, and then `yield`s the new state as a tuple. Using `yield` makes this a generator, which is memory-efficient as it produces the new states one by one on demand.

In [ ]:
def solve_puzzle(start, goal):
    """BFS guarantees the fewest moves solution."""
    queue = deque([start])
    parents = {start: None}

    while queue:
        current = queue.popleft()
        if current == goal:
            break
        for nxt in neighbors(current):
            if nxt not in parents:
                parents[nxt] = current
                queue.append(nxt)

    return parents

### Code: The BFS Solver

This function, `solve_puzzle`, implements the BFS algorithm.

- **`queue`**: A `deque` (double-ended queue) is used for the frontier. `deque` is highly efficient for adding (`append`) and removing (`popleft`) elements from both ends.
- **`parents`**: A dictionary that serves two purposes:
    1.  It keeps track of all visited states, preventing cycles.
    2.  It stores the path by mapping each state to the state that came before it (`{child: parent}`).

#### The Loop

The `while queue:` loop drives the search.
1.  It dequeues the state at the front of the queue (`current`).
2.  It checks if `current` is the `goal`. If so, the search is complete.
3.  If not, it calls `neighbors(current)` to get all possible next states.
4.  For each `nxt` state, it checks if it is already in `parents`. If not, it means we haven't visited this state before.
5.  The unvisited neighbor is then added to the `parents` map (with `current` as its parent) and enqueued for future exploration.

In [ ]:
def reconstruct_moves(parents, start, goal):
    if goal not in parents:
        return []
    path = []
    current = goal
    while current is not None:
        path.append(current)
        current = parents[current]
    path.reverse()
    return path

### Code: Reconstructing the Path

After the `solve_puzzle` function finds the goal, the `parents` dictionary holds the key to the solution path. The `reconstruct_moves` function uses this dictionary to trace the path backward from the goal to the start.

1.  It starts with the `goal` state.
2.  It repeatedly looks up the current state in the `parents` dictionary to find its predecessor.
3.  Each state is added to the `path` list.
4.  This continues until it reaches the `start_state`, whose parent is `None`.
5.  Finally, the path is reversed to get the correct sequence of moves from start to finish.

In [ ]:
parents = solve_puzzle(start_state, goal_state)
sequence = reconstruct_moves(parents, start_state, goal_state)

if not sequence:
    print("No solution found.")
else:
    print(f"Solved in {len(sequence) - 1} moves. States along the way:")
    for step, state in enumerate(sequence):
        print(f"\nStep {step}:")
        show_state(state)

### Conclusion: Solving and Displaying

This final cell brings everything together to solve the puzzle and display the solution.

1.  It calls `solve_puzzle` to perform the BFS search, which returns the `parents` map.
2.  It calls `reconstruct_moves` to build the sequence of states from start to goal.
3.  If a sequence is found, it prints the total number of moves and then iterates through the path, printing each step of the solution clearly.
4.  If the sequence is empty, it means the `goal` was never reached, and it reports that no solution was found.